In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df
    
def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

## Instalando as Dependências do Ubuntu

In [2]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500

# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300

# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium

## Configuração do Web-Driver

In [3]:
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.headless = True

from selenium.webdriver.chrome.service import Service
wd_Chrome = webdriver.Chrome("/usr/bin/chromedriver", options=options)

## Iniciando a Raspagem de Dados

In [4]:
# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com/")

In [5]:
# Fechando Botão de Cookies
try:
    button_cookies = wd_Chrome.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    pass

# Selecionando o Dia de Amanhã
wd_Chrome.find_element(By.CSS_SELECTOR,'button.calendar__navigation--tomorrow').click()
time.sleep(3)

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odd_H':[],'Odd_D':[],'Odd_A':[],'Odd_Over15':[],'Odd_Under15':[],'Odd_Over25':[],'Odd_Under25':[],'Odd_BTTS_Yes':[],'Odd_BTTS_No':[]}

In [6]:
# Pegando as Informacoes Básicas do Jogo
for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
    
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    except:
        pass

# Match Odds
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
        time.sleep(2)
        
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                Odd_H = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odd_D = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text) 
                Odd_A = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text)
    except:
        pass

# Over e Under 2.5
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/over-under/full-time')
        time.sleep(2)
        
        linhas_over_under = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table.oddsCell__odds')
        for i in linhas_over_under:
            linhas = i.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
            for linha in linhas:
                try:
                    qtd_gols = linha.find_element(By.CSS_SELECTOR,'span.oddsCell__noOddsCell').text.replace('.','')
                    if qtd_gols == '05':
                        Odd_Over05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                    if qtd_gols == '15':
                        Odd_Over15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                        #print(Odd_Under15)
                    if qtd_gols == '25':
                        Odd_Over25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                except:
                    pass
    except:
        pass

# Ambas Marcam (BTTS)
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/both-teams-to-score/full-time')
        time.sleep(2)
    
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                BTTS_Yes = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                BTTS_No = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
    except:
        pass    
    
    try:
        print(Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,BTTS_Yes,BTTS_No) 

        jogo['Date'].append(Date)
        jogo['Time'].append(Time)
        jogo['Country'].append(Country)
        jogo['League'].append(League)
        jogo['Home'].append(Home)
        jogo['Away'].append(Away)
        jogo['Odd_H'].append(Odd_H)
        jogo['Odd_D'].append(Odd_D)
        jogo['Odd_A'].append(Odd_A)
        jogo['Odd_Over15'].append(Odd_Over15)
        jogo['Odd_Under15'].append(Odd_Under15)
        jogo['Odd_Over25'].append(Odd_Over25)
        jogo['Odd_Under25'].append(Odd_Under25)
        jogo['Odd_BTTS_Yes'].append(BTTS_Yes)
        jogo['Odd_BTTS_No'].append(BTTS_No)

    except:
        pass

  0%|          | 0/54 [00:00<?, ?it/s]

  2%|▏         | 1/54 [00:10<09:15, 10.47s/it]

19.06.2023 13:00 EUROPE EURO - QUALIFICATION - ROUND 4 Armenia Latvia 1.75 4.0 4.5 1.28 3.75 1.83 2.0 1.75 2.05


  4%|▎         | 2/54 [00:20<08:44, 10.10s/it]

19.06.2023 13:00 EUROPE EURO - QUALIFICATION - ROUND 4 Finland San Marino 1.02 21.0 126.0 1.04 11.0 1.19 4.75 4.0 1.24


...

19.06.2023 12:30 WORLD CLUB FRIENDLY O. Ljubljana (Slo) Radomlje (Slo) 3.1 3.5 2.1 1.22 4.0 1.72 2.05 1.62 2.2


 98%|█████████▊| 53/54 [07:06<00:07,  7.72s/it]

19.06.2023 12:30 WORLD CLUB FRIENDLY Rogaska (Slo) Ferencvaros (Hun) 13.0 7.0 1.16 1.07 7.0 1.36 3.0 1.95 1.75


100%|██████████| 54/54 [07:13<00:00,  8.03s/it]

19.06.2023 06:00 WORLD FRIENDLY INTERNATIONAL WOMEN Poland U23 W Vietnam W 13.0 7.0 1.16 1.07 7.0 1.36 3.0 1.95 1.75


## Gerando DataFrame com os Jogos do Dia

In [7]:
df = pd.DataFrame(jogo)
df = drop_reset_index(df)
df

,Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,Odd_BTTS_Yes,Odd_BTTS_No
1,19.06.2023,13:00,EUROPE,EURO - QUALIFICATION - ROUND 4,Armenia,Latvia,1.75,4.00,4.50,1.28,3.75,1.83,2.00,1.75,2.05
2,19.06.2023,13:00,EUROPE,EURO - QUALIFICATION - ROUND 4,Finland,San Marino,1.02,21.00,126.00,1.04,11.00,1.19,4.75,4.00,1.24
3,19.06.2023,13:00,EUROPE,EURO - QUALIFICATION - ROUND 4,Ukraine,Malta,1.09,8.50,21.00,1.10,7.00,1.36,3.00,2.25,1.63
4,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,Belarus,Kosovo,5.50,3.75,1.70,1.40,2.95,2.25,1.66,2.15,1.70
5,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,England,North Macedonia,1.11,9.50,29.00,1.13,6.00,1.44,2.80,2.60,1.48
6,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,France,Greece,1.26,6.00,13.00,1.18,5.00,1.58,2.40,2.05,1.75
7,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,Ireland,Gibraltar,25.00,65.00,51.00,1.06,9.50,1.24,4.20,4.33,1.20
8,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,Israel,Andorra,1.12,9.50,26.00,1.13,6.00,1.44,2.85,2.45,1.53
9,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,Northern Ireland,Kazakhstan,1.63,3.75,6.50,1.40,2.95,2.25,1.66,2.20,1.65
10,19.06.2023,15:45,EUROPE,EURO - QUALIFICATION - ROUND 4,Slovenia,Denmark,4.33,3.60,1.88,1.40,3.00,2.15,1.70,2.00,1.80


In [8]:
df = df.rename(columns={'Odd_H': 'FT_Odd_H',
                        'Odd_D': 'FT_Odd_D',
                        'Odd_A': 'FT_Odd_A',
                        'Odd_Over15': 'FT_Odd_Over15',
                        'Odd_Under15': 'FT_Odd_Under15',
                        'Odd_Over25': 'FT_Odd_Over25',
                        'Odd_Under25': 'FT_Odd_Under25',
                        'Odd_BTTS_Yes': 'FT_Odd_BTTS_Yes',
                        'Odd_BTTS_No': 'FT_Odd_BTTS_No',
                        })

In [9]:
# Gerando os dados para excel
from datetime import date, datetime, timedelta
data = datetime.today() + timedelta(1)
amanha = data.strftime('%Y-%m-%d')
nome = 'Jogos_do_Dia_FlashScore.xlsx'

df.to_excel(f'G:\\Meu Drive\\Programacao\\futPythonTrader\\Jogos do dia\\{amanha}_{nome}', index = False)